# Introduction

This notebook serves two primary purposes. First, it demonstrates Gretel's ability to generate realistic data from a provided schema. We will focus on creating synthetic customer bank transaction data, reflecting the type of data a customer might provide while ensuring privacy and avoiding the use of real sensitive information.

The second purpose of this notebook is to create a synthetic dataset that can be used in a later notebook for showcasing data privacy differential techniques. This generated data will act as a stand-in for real sensitive data, allowing us to explore privacy-preserving methods without compromising actual customer information.

We will generate a dataset with the following schema, typical of financial transaction data:

*   customer_id
*   transaction_date
*   transaction_amount
*   transaction_type
*   transaction_category
*   account\_balance

Gretel will be used to generate realistic data based on this schema, including the nuanced relationship between transactions and account balances. This will be achieved by generating the core transaction data first and then enriching it to calculate running account balances, showcasing Gretel's flexibility and ability to model complex relationships.

## Prerequisites

In [1]:
!pip install -U --quiet gretel-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.2/424.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.27.0 requires tabul

In [2]:
!gretel configure


Gretel.ai COPYRIGHT Notice


The Gretel CLI and Python SDK, installed through the "gretel-client"
package or other mechanism is free and open source software under
the Apache 2.0 License.

When using the CLI or SDK, you may launch "Gretel Worker(s)"
that are hosted in your local environment as containers. These
workers are launched automatically when running commands that create
models or process data records.

The "Gretel Worker" and all code within it is copyrighted and an
extension of the Gretel Service and licensed under the Gretel.ai
Terms of Service.  These terms can be found at https://gretel.ai/terms
section G paragraph 2.



Endpoint [https://api.gretel.cloud]: 
Artifact Endpoint [cloud]: 
Default Runner (cloud, local, hybrid) [cloud]: 
Gretel API Key [None]: 
Default Project [none]: 
Using endpoint https://api.gretel.cloud
Logged in as ericphamhung@gmail.com ✅
INFO: Configuration written to /root/.gretel/config.json. Done.
{
    "endpoint": "https://api.gretel.cloud",
    "a

In [3]:
!gretel whoami

{
    "email": "ericphamhung@gmail.com",
    "config": {
        "endpoint": "https://api.gretel.cloud",
        "artifact_endpoint": "cloud",
        "api_key": "grtu6836****",
        "default_project_name": null,
        "default_runner": "cloud",
        "preview_features": "disabled"
    }
}


# Main

This section outlines the core steps involved in generating and enriching synthetic customer bank transaction data. We will leverage Gretel's Tabular Data API to create realistic data and ensure privacy. The workflow consists of the following stages:

*   **Data Generation:** We will utilize Gretel's Tabular Data API to generate synthetic bank transactions based on a predefined schema. This will involve defining a prompt that outlines the desired data structure and characteristics, including customer IDs, transaction dates, amounts, types, and categories.


*   **Account Balance Calculation:** After generating the transaction data, we will enrich it by calculating running account balances for each customer. This step will involve extracting unique customer IDs, assigning starting balances, and applying a function to compute the balance for each transaction based on its type and amount.

*   **Data Verification and Export:** Finally, we will verify the generated data to ensure its accuracy and consistency. We will examine transactions for specific customers to confirm the correctness of the account balance calculations. Once verified, the data will be exported to a CSV file for further use or analysis.

By following this workflow, we aim to simulate a sensitive dataset that a customer might have for the use of as model training or testing.



## Data Generation

In [4]:
# Import the pandas library for data manipulation
# Import the Gretel library for data generation and transformation

import pandas as pd
from gretel_client import Gretel

# Initialize the Gretel client with your API key
gretel = Gretel(api_key="prompt", validate=True)

Found cached Gretel credentials
Using endpoint https://api.gretel.cloud
Logged in as ericphamhung@gmail.com ✅


In [5]:
# Get a list of available tabular data models
print(gretel.factories.get_navigator_model_list("tabular"))

['gretelai/auto', 'gretelai/Mistral-7B-Instruct-v0.2/industry', 'gretelai/Llama-3.1-8B-Instruct', 'gretelai-azure/gpt-3.5-turbo', 'gretelai-google/gemini-pro']


In [6]:
# Initialize the Gretel tabular data API
tabular = gretel.factories.initialize_navigator_api("tabular", backend_model="gretelai/auto")

# Define a prompt to generate customer bank transaction data
# Notice that we do not include "account_balance" in the genesis of the data
prompt = """\
Generate customer bank transaction data. Ensure multiple transactions rows for the same customer_ids. Ensure there are multiple transactions in the same day. Limit to 25 unique customers.

Include the following columns:
● customer_id (in the form of CUST_XXX)
● transaction_date (between 2022-01-01 to 2024-12-31)
● transaction_amount
● transaction_type (only as "credit" or "debit")
● transaction_category
"""

# Generate tabular data from the prompt
df = tabular.generate(prompt, num_records=1000)

Backend model: gretelai/auto
API path: https://api.gretel.cloud/v1/inference/tabular/
Navigator Tabular initialized 🚀


Generating records: 100%|██████████| 1000/1000 [09:42, 1.72 records/s]


In [7]:
# Display the first few rows of the generated data
df.head()

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category
0,CUST_001,2022-01-01,100.0,credit,rent
1,CUST_001,2022-01-01,50.0,debit,groceries
2,CUST_001,2022-01-02,75.0,credit,salary
3,CUST_001,2022-01-02,25.0,debit,entertainment
4,CUST_001,2022-01-03,150.0,credit,investment


In [8]:
df.to_csv('transactions_only.csv', index=False)

## Account Balance Calculation

In [9]:
# Extract unique customer IDs
df_unique_customers = df['customer_id'].drop_duplicates().to_frame()

# Define a prompt to add a "starting_amount" column
edit_prompt = """\
Add the following column to the provided table:

- starting_amount
"""

# Add the starting_amount column to the unique customer data
df_starting = tabular.edit(edit_prompt, seed_data=df_unique_customers)

Editing records: 100%|██████████| 73/73 [00:38, 1.90 records/s]


In [10]:
# Display the data with the starting amount column
df_starting

,customer_id,starting_amount
0,CUST_001,1000.0
1,CUST_002,500.0
2,CUST_003,800.0
3,CUST_004,1200.0
4,CUST_005,300.0
...,...,...
68,CUST_069,380.0
69,CUST_070,670.0
70,CUST_071,580.0
71,CUST_072,810.0


In [11]:
df_starting.to_csv('starting_amounts.csv', index=False)

In [24]:
df_starting['starting_amount'] = df_starting['starting_amount'].fillna(0)

In [25]:
# Merge the transaction data with the starting amount data
merged_table = pd.merge(df, df_starting, on="customer_id", how='left')
merged_table.head()

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount
0,CUST_001,2022-01-01,100.0,credit,rent,1000.0
1,CUST_001,2022-01-01,50.0,debit,groceries,1000.0
2,CUST_001,2022-01-02,75.0,credit,salary,1000.0
3,CUST_001,2022-01-02,25.0,debit,entertainment,1000.0
4,CUST_001,2022-01-03,150.0,credit,investment,1000.0


In [29]:
merged_table

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount
0,CUST_001,2022-01-01,100.0,credit,rent,1000.0
1,CUST_001,2022-01-01,50.0,debit,groceries,1000.0
2,CUST_001,2022-01-02,75.0,credit,salary,1000.0
3,CUST_001,2022-01-02,25.0,debit,entertainment,1000.0
4,CUST_001,2022-01-03,150.0,credit,investment,1000.0
...,...,...,...,...,...,...
995,CUST_048,2023-01-17,1210.0,credit,salary,0.0
996,CUST_049,2023-01-18,1000.0,debit,groceries,0.0
997,CUST_049,2023-01-18,1230.0,credit,salary,0.0
998,CUST_050,2023-01-19,1020.0,debit,dining,0.0


In [30]:
# Define a function to compute the account balance
def compute_balance(row, balances):
    previous_balance = balances.get(row['customer_id'], row['starting_amount'])
    transaction_effect = row['transaction_amount'] if row['transaction_type'] == "credit" else -row['transaction_amount']
    current_balance = previous_balance + transaction_effect
    balances[row['customer_id']] = current_balance
    return current_balance

# Compute the account balance for each transaction
balances = {}
merged_table['account_balance'] = merged_table.apply(compute_balance, axis=1, balances=balances)


In [31]:
# Display the data with the account balance column
merged_table

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount,account_balance
0,CUST_001,2022-01-01,100.0,credit,rent,1000.0,1100.0
1,CUST_001,2022-01-01,50.0,debit,groceries,1000.0,1050.0
2,CUST_001,2022-01-02,75.0,credit,salary,1000.0,1125.0
3,CUST_001,2022-01-02,25.0,debit,entertainment,1000.0,1100.0
4,CUST_001,2022-01-03,150.0,credit,investment,1000.0,1250.0
...,...,...,...,...,...,...,...
995,CUST_048,2023-01-17,1210.0,credit,salary,0.0,11470.0
996,CUST_049,2023-01-18,1000.0,debit,groceries,0.0,-14890.0
997,CUST_049,2023-01-18,1230.0,credit,salary,0.0,-13660.0
998,CUST_050,2023-01-19,1020.0,debit,dining,0.0,13100.0


## Data Verification and Export

In [32]:
# Display transactions for a specific customer
merged_table[merged_table['customer_id']=='CUST_001']

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount,account_balance
0,CUST_001,2022-01-01,100.00,credit,rent,1000.0,1100.00
1,CUST_001,2022-01-01,50.00,debit,groceries,1000.0,1050.00
2,CUST_001,2022-01-02,75.00,credit,salary,1000.0,1125.00
3,CUST_001,2022-01-02,25.00,debit,entertainment,1000.0,1100.00
4,CUST_001,2022-01-03,150.00,credit,investment,1000.0,1250.00
5,CUST_001,2022-01-03,20.00,debit,transportation,1000.0,1230.00
6,CUST_001,2022-01-04,80.00,credit,loan,1000.0,1310.00
100,CUST_001,2022-01-01,2500.00,credit,salary,1000.0,3810.00
101,CUST_001,2022-01-01,120.00,debit,groceries,1000.0,3690.00
102,CUST_001,2022-01-02,800.00,credit,rent,1000.0,4490.00


In [33]:
# Save the data to a CSV file
merged_table.to_csv('transactions_merged.csv', index=False)